In [ ]:
# Retrieve DHIS2 data for a specified time range
# using Python wrapper and export to excel file.
# by Ali Tobah - atobah@gmail.com

# Resources:
# https://pypi.org/project/dhis2.py/
# https://docs.dhis2.org/en/develop/using-the-api/dhis-core-version-240/introduction.html
# https://play.dhis2.org/40.2.0/api/resources

# Only need to run this once for each session.
!pip install dhis2.py

In [ ]:
import pandas as pd
import getpass
from dhis2 import Api

In [ ]:
def retrieveNameID(dataParam):
    '''
        Input: Parameter for the API request.
        Output: Prints DF shape and head.
        Return: Dataframe.
    '''
    # Run a request on an API object to get data parameter names
    # and IDs. The request returns a dictionary of one key with a
    # list of dictionaries as its value.
    response = api.get_paged(dataParam, merge=True)

    # Read the data and feed into a dataframe
    dhisData = response[dataParam]
    dataDF = pd.DataFrame(dhisData)

    # Rename columns to unique names and print
    colName = dataParam[:-1] + 'Name'
    colID = dataParam[:-1] + 'ID'
    dataDF = dataDF.rename(columns={'displayName': colName, 'id': colID})

    print("Dataframe shape: ", dataDF.shape, '\n')
    print(dataDF.head())

    return dataDF

In [ ]:
# Create an API object for the demo data.

# Prompt for URL where data is located.
# For demo data, use "play.dhis2.org/demo"
dhisURL = input("Data location (URL): ")

# Prompt for username and password while creating the object.
# User name is "admin" for demo data
# Password is "district" for demo data. Use getpass to protect it.
api = Api(dhisURL, input("Username: "), getpass.getpass(prompt="Password: "))

Data location (URL): play.dhis2.org/demo
Username: admin
Password: ··········


In [ ]:
# Get dataset names and IDs dataframe.
# Create a list for use in the request.
dataSetDF = retrieveNameID('dataSets')
dataSetIDList = list(dataSetDF.dataSetID)

Dataframe shape:  (27, 2) 

                      dataSetName    dataSetID
0             ART monthly summary  lyLU2wR22tC
1                    Child Health  BfMAe6Itzgt
2  Clinical Monitoring Checklist   VTdjfLXXmoi
3                       EPI Stock  TuL8IOPzpHh
4              Emergency Response  Lpw6GcnTrmS


In [ ]:
# Get organisational unit names and IDs dataframe.
# Create a list for use in the request.
# Create a dictionary to map IDs to names in final result.
orgUnitDF = retrieveNameID('organisationUnits')
orgUnitIDList = list(orgUnitDF.organisationUnitID)
orgUnitDict = orgUnitDF.set_index('organisationUnitID')['organisationUnitName'].to_dict()

Dataframe shape:  (1333, 2) 

  organisationUnitName organisationUnitID
0         Panderu MCHP        ueuQlqb8ccl
1               Admiro        bF6hxdG28kg
2          Adonkia CHP        Rp268JB6Ne4
3     Afro Arab Clinic        cDw53Ej8rju
4            Agape CHP        GvFqTavdpGE


In [ ]:
# Get data element names and IDs dataframe.
# Create a dictionary to map IDs to names in final result.
dataElementDF = retrieveNameID('dataElements')
dataElementDict = dataElementDF.set_index('dataElementID')['dataElementName'].to_dict()

Dataframe shape:  (1162, 2) 

       dataElementName dataElementID
0       _Order_Reverse   deeu8rjsqvH
1       _Order_Forward   FEkGksxhOpH
2   _Order_Last Survey   iW2zVNwfDK6
3    _Stats_created_on   rcAkFQFYknk
4        ANC 1st visit   fbfJHSPpUQD


In [ ]:
# Get category option names and IDs dataframe.
# Create a dictionary to map IDs to names in final result.
categoryOptiontDF = retrieveNameID('categoryOptionCombos')
categOptDict = categoryOptiontDF.set_index('categoryOptionComboID')['categoryOptionComboName'].to_dict()

Dataframe shape:  (225, 2) 

  categoryOptionComboName categoryOptionComboID
0                   0-11m           S34ULMcHMca
1                   0-11m           sqGRzCziswD
2                    0-4y           o2gxEt6Ek2C
3                  12-59m           LEDQQXEpWUl
4                  12-59m           wHBMVthqIX4


In [ ]:
# Prompt for start and end dates.
# When testing, use 2022-01-01 and 2022-01-31
startDate = input("Start date (yyyy-mm-dd): ")
endDate = input("End date (yyyy-mm-dd): ")

Start date (yyyy-mm-dd): 2022-01-01
End date (yyyy-mm-dd): 2022-01-31


In [ ]:
# Request data for all datasets in dataset list,
# and all organisational units in organisational unit list,
# for given start and end dates.

# Since data is too large for one request, break organisational
# unit ID list into a list of lists of organisational units, each
# of size 100.
orgNestList = [orgUnitIDList[i:i + 100] for i in range(0, len(orgUnitIDList), 100)]

# Create an empty list to hold the responses.
# This will be a list of dictionaries, each dictionary
# representing one data element, as seen in the next cell.
responseList = []
i=0    # List counter for user convenience
print("Expect", len(orgNestList), "iterations:")

# Retrieve data for each 100 orgUnits and add to
# response list using extend (as opposed to append).
for eaOrgList in orgNestList:
    response = api.get('dataValueSets', params={
        'dataSet': dataSetIDList,
        'orgUnit': eaOrgList,
        'startDate': startDate,
        'endDate': endDate
    })
    addnlElements = response.json()['dataValues']
    responseList.extend(addnlElements)

    # Increment counter and print for user patience
    i+=1
    print(i, "..", len(addnlElements), "data elements")

print("Total data elements:", len(responseList))


Expect 14 iterations:
1 .. 12077 data elements
2 .. 10231 data elements
3 .. 12257 data elements
4 .. 12151 data elements
5 .. 11497 data elements
6 .. 11315 data elements
7 .. 12706 data elements
8 .. 12112 data elements
9 .. 13396 data elements
10 .. 12190 data elements
11 .. 12657 data elements
12 .. 10773 data elements
13 .. 11739 data elements
14 .. 4034 data elements
Total data elements: 159135


In [ ]:
# Print first value in the list (i.e the first data element)
# for visual confirmation.
responseList[0]

{'dataElement': 'd9vZ3HOlzAd',
 'period': '202201',
 'orgUnit': 'aF6iPGbrcRk',
 'categoryOptionCombo': 'KPP63zJPkOu',
 'attributeOptionCombo': 'HllvX50cXC0',
 'value': '97',
 'storedBy': '',
 'created': '2014-03-02T21:31:02.000+0000',
 'lastUpdated': '2014-03-02T21:31:02.000+0000',
 'comment': '',
 'followup': False}

In [ ]:
# Feed all data elements from the list into a dataframe.
dhisDF = pd.DataFrame(responseList)
print("Dataframe shape: ", dhisDF.shape, '\n')
dhisDF.head()

Dataframe shape:  (159135, 11) 



,dataElement,period,orgUnit,categoryOptionCombo,attributeOptionCombo,value,storedBy,created,lastUpdated,comment,followup
0,d9vZ3HOlzAd,202201,aF6iPGbrcRk,KPP63zJPkOu,HllvX50cXC0,97,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
1,dY4OCwl0Y7Y,202201,aF6iPGbrcRk,KPP63zJPkOu,HllvX50cXC0,85,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
2,BXgDHhPdFVU,202201,aF6iPGbrcRk,KPP63zJPkOu,HllvX50cXC0,80,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
3,TCfIC3NDgQK,202201,aF6iPGbrcRk,KPP63zJPkOu,HllvX50cXC0,96,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
4,hJNC4Bu2Mkv,202201,aF6iPGbrcRk,KPP63zJPkOu,HllvX50cXC0,87,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False


In [ ]:
# Replace IDs with names using dictionaries from above.
# Note that the attributeOptionCombo uses the same
# dictionary as categoryOptionCombo. This is not a typo.
dhisDF = dhisDF.copy()
dhisDF['dataElement'] = dhisDF['dataElement'].map(dataElementDict)
dhisDF['orgUnit'] = dhisDF['orgUnit'].map(orgUnitDict)
dhisDF['categoryOptionCombo'] = dhisDF['categoryOptionCombo'].map(categOptDict)      # Same dictionary.
dhisDF['attributeOptionCombo'] = dhisDF['attributeOptionCombo'].map(categOptDict)    # Not a typo.
print("Dataframe shape: ", dhisDF.shape, '\n')
dhisDF.head()

Dataframe shape:  (159135, 11) 



,dataElement,period,orgUnit,categoryOptionCombo,attributeOptionCombo,value,storedBy,created,lastUpdated,comment,followup
0,Commodities - Antenatal Corticosteroids,202201,Bandasuma Fiama MCHP,Quantity to be ordered,default,97,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
1,Commodities - Female Condoms,202201,Bandasuma Fiama MCHP,Quantity to be ordered,default,85,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
2,Commodities - Emergency Contraception,202201,Bandasuma Fiama MCHP,Quantity to be ordered,default,80,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
3,Commodities - Zinc,202201,Bandasuma Fiama MCHP,Quantity to be ordered,default,96,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False
4,Commodities - Oxytocin,202201,Bandasuma Fiama MCHP,Quantity to be ordered,default,87,,2014-03-02T21:31:02.000+0000,2014-03-02T21:31:02.000+0000,,False


In [ ]:
# Export to an excel file

newStartDate = startDate.replace('-', '')
newEndDate = endDate.replace('-', '')
dhisFileName = "DHIS-Demo-Data" + "-" + newStartDate + "-" + newEndDate + ".xlsx"
print("Exporting ...")
dhisDF.to_excel(dhisFileName)
print("Data exported to:", dhisFileName)

Exporting ...
Data exported to: DHIS-Demo-Data-20220101-20220131.xlsx
